In [107]:
import pandas as pd

df = pd.read_csv('../relevant csvs/frequency for correlation2.csv')
df = df.dropna()
num_columns = df.shape[1]

df=df.dropna()
df = df.drop_duplicates()
df = df.groupby(['Month','Latitude', 'Longitude','Power (J)count','Power (J)mean', 'Sodium','Temperature','pH','Carbon','Alkalinity'], as_index=False).agg({'Precipitation': "sum"})
df = df[~df['Month'].str.startswith('2020')]
df = df[df['Power (J)count']>0]
df["Month"] = df["Month"].apply(lambda x: x.split("-")[1])
df

,Month,Latitude,Longitude,Power (J)count,Power (J)mean,Sodium,Temperature,pH,Carbon,Alkalinity,Precipitation
8,12,31.0,17.0,1.0,1747.330000,38.352597,20.059435,8.094838,2.342358,2.694373,20.079572
14,12,31.0,26.0,1.0,21586.630000,38.601412,19.575738,8.098050,2.350928,2.702863,20.579652
15,12,31.0,27.0,13.0,10764.926154,38.697234,19.879898,8.094135,2.351454,2.704876,20.025204
16,12,31.0,28.0,38.0,10836.028684,38.772010,20.552371,8.084135,2.350173,2.704298,18.068275
17,12,31.0,29.0,48.0,12794.811667,38.836603,20.770606,8.083319,2.348381,2.704870,28.008409
...,...,...,...,...,...,...,...,...,...,...,...
10504,12,44.0,14.0,225.0,721.654533,38.316125,16.414821,8.136798,2.378079,2.723378,121.439349
10505,12,44.0,15.0,43.0,680.203721,37.957863,15.387796,8.147993,2.374119,2.712510,88.667070
10506,12,45.0,12.0,1.0,47.270000,36.661519,13.043289,8.194665,2.459669,2.803127,147.647299
10507,12,45.0,13.0,8.0,71.916250,37.700932,14.229338,8.165619,2.329798,2.661838,177.932748


In [108]:
#month to dummy vars

dummy_variables = pd.get_dummies(df['Month'], prefix='Month')

df = pd.concat([df, dummy_variables], axis=1)

df["alk/sal"] = df["Alkalinity"] / df["Sodium"]

import numpy as np

# Assuming your DataFrame is named df
df['Count'] = np.where(df['Power (J)count'] != 0, np.log(df['Power (J)count']), 0)


df


,Month,Latitude,Longitude,Power (J)count,Power (J)mean,Sodium,Temperature,pH,Carbon,Alkalinity,Precipitation,Month_01,Month_02,Month_12,alk/sal,Count
8,12,31.0,17.0,1.0,1747.330000,38.352597,20.059435,8.094838,2.342358,2.694373,20.079572,0,0,1,0.070253,0.000000
14,12,31.0,26.0,1.0,21586.630000,38.601412,19.575738,8.098050,2.350928,2.702863,20.579652,0,0,1,0.070020,0.000000
15,12,31.0,27.0,13.0,10764.926154,38.697234,19.879898,8.094135,2.351454,2.704876,20.025204,0,0,1,0.069898,2.564949
16,12,31.0,28.0,38.0,10836.028684,38.772010,20.552371,8.084135,2.350173,2.704298,18.068275,0,0,1,0.069749,3.637586
17,12,31.0,29.0,48.0,12794.811667,38.836603,20.770606,8.083319,2.348381,2.704870,28.008409,0,0,1,0.069647,3.871201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10504,12,44.0,14.0,225.0,721.654533,38.316125,16.414821,8.136798,2.378079,2.723378,121.439349,0,0,1,0.071077,5.416100
10505,12,44.0,15.0,43.0,680.203721,37.957863,15.387796,8.147993,2.374119,2.712510,88.667070,0,0,1,0.071461,3.761200
10506,12,45.0,12.0,1.0,47.270000,36.661519,13.043289,8.194665,2.459669,2.803127,147.647299,0,0,1,0.076460,0.000000
10507,12,45.0,13.0,8.0,71.916250,37.700932,14.229338,8.165619,2.329798,2.661838,177.932748,0,0,1,0.070604,2.079442


In [ ]:
#LinearRegression code for building a model with specific features
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest, f_regression

max_accuracy = 0
best_features = []

# Load the data into a Pandas dataframe
df = pd.read_csv("../RELEVANT CSVS/frequency for correlation.csv")


# Define the target variable and the features you want to use for prediction
target = 'Count'
features = ['Latitude', 'Longitude', 'Acidity', 'Alkalinity', 'Carbon', 'Precipitation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], np.log(df[target]), test_size=0.3, random_state=42)

# Use SelectKBest to select the top k features based on f_regression score
selector = SelectKBest(f_regression, k=6)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance using R-squared metric
r2 = r2_score(y_test, y_pred)

print(r2)



In [110]:
#LinearRegression code that tries all possible combinations of features and prints the best combination

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity','Precipitation' ,'Carbon','Month_01','Month_02','Month_12','alk/sal']

max_r2 = 0
best_features = []
rmse_train2 = 0


# Generate all possible combinations of features
for r in range(1, len(all_features) + 1):
    feature_combinations = combinations(all_features, r)
    for features in feature_combinations:
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(df[list(features)], df[target], test_size=0.3, random_state=42)

        # Create a linear regression model
        model = LinearRegression()

        # Fit the model to the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model's performance using R-squared metric
        r2 = r2_score(y_test, y_pred)
        rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        y_pred_train = model.predict(X_train)
        rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

        # Update the best features and max R-squared score if necessary
        if r2 > max_r2:
            max_r2 = r2
            best_features_r2 = features
            rmse2= rmse_test
            rmse_train1 = rmse_train

# Print the best features and the corresponding R-squared score
print("Features:", best_features_r2)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(rmse2))
print("RMSE on train set: {:.6f}".format(rmse_train))


Features: ('Latitude', 'Longitude', 'pH', 'Alkalinity', 'Precipitation', 'Carbon', 'Month_02', 'Month_12')
R-squared Score: 0.2205343618089124
RMSE on test set: 251.764474
RMSE on train set: 250.516284


In [ ]:
#DecisionTree code for building a model with specific features

from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Acidity', 'Alkalinity', 'Carbon', 'Precipitation', 'Month_01', 'Month_12']

max_r2 = 0
best_features = []

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[list(all_features)], np.log(df[target]), test_size=0.3, random_state=42)


# Create a decision tree regression model
model = DecisionTreeRegressor(max_depth=8)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance using R-squared metric
r2 = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
y_pred_train = model.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

if r2 > max_r2:
    max_r2 = r2
    best_features = features
    best_rmse= rmse_test
    best_rmse_train = rmse_train

print("Best Features for max r2:", best_features)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(best_rmse))
print("RMSE on train set: {:.6f}".format(best_rmse_train))
print("----------------------------------------------")


In [112]:
#LinearRegression code that tries all possible combinations of features and prints the best combination as well as all combinations with r2<0.346

from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity','Precipitation' ,'Carbon','Month_01','Month_02','Month_12','alk/sal']

max_r2 = 0
best_features = []

# Generate all possible combinations of features
for r in range(1, len(all_features) + 1):
    feature_combinations = combinations(all_features, r)
    for features in feature_combinations:
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(df[list(features)], df[target], test_size=0.3, random_state=42)

        
        # Create a decision tree regression model
        model = DecisionTreeRegressor(max_depth=8)

        # Fit the model to the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model's performance using R-squared metric
        r2 = r2_score(y_test, y_pred)
        rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        y_pred_train = model.predict(X_train)
        rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

        # Update the best features and max R-squared score if necessary
        if r2 > max_r2:
            max_r2 = r2
            best_features_r2 = features
            rmse2= rmse_test
            rmse_train1 = rmse_train

# Print the best features and the corresponding R-squared score
print("Features:", best_features_r2)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(rmse2))
print("RMSE on train set: {:.6f}".format(rmse_train))


Features: ('Latitude', 'Longitude', 'Month_02')
R-squared Score: 0.36724330880262057
RMSE on test set: 241.831675
RMSE on train set: 206.975405


In [ ]:
#ReandomForest specific features
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Salinity', 'Temperature', 'Acidity', 'Alkalinity', 'Carbon', 'Precipitation','Month_01','Month_02','Month_12']

max_r2 = 0
best_features = []

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[list(all_features)], np.log(df[target]), test_size=0.3, random_state=42)

# Create a random forest regression model
model = RandomForestRegressor(n_estimators=100, max_depth=9)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's performance using R-squared metric
r2 = r2_score(y_test, y_pred)
rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
y_pred_train = model.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

if r2 > max_r2:
    max_r2 = r2
    best_features = all_features
    best_rmse = rmse_test
    best_rmse_train = rmse_train

print("Best Features for max r2:", best_features)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(best_rmse))
print("RMSE on train set: {:.6f}".format(best_rmse_train))
print("----------------------------------------------")


In [113]:
#RandomForest only combinations of 9 features or more

from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity','Precipitation' ,'Carbon','Month_01','Month_02','Month_12','alk/sal']

max_r2 = 0
best_features = []

# Generate all possible combinations of features
for r in range(9, len(all_features) + 1):
    feature_combinations = combinations(all_features, r)
    for features in feature_combinations:
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(df[list(features)], df[target], test_size=0.3, random_state=42)

        # Create a random forest regression model
        model = RandomForestRegressor(n_estimators=100, max_depth=r)

        # Fit the model to the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model's performance using R-squared metric
        r2 = r2_score(y_test, y_pred)
        rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        y_pred_train = model.predict(X_train)
        rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

        if r2 > 0.4:
            print("Features:", features)
            print("R-squared Score:", r2)
            print("RMSE on test set: {:.6f}".format(rmse_test))
            print("RMSE on train set: {:.6f}".format(rmse_train))
            print("+++++++++++++++++++++++++++++++++++++++++++++++")
        
        # Update the best features and max R-squared score if necessary
        if r2 > max_r2:
            max_r2 = r2
            best_features_r2 = features
            rmse2= rmse_test
            rmse_train1 = rmse_train

# Print the best features and the corresponding R-squared score
print("Features:", best_features_r2)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(rmse2))
print("RMSE on train set: {:.6f}".format(rmse_train))



Features: ('Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity', 'Precipitation', 'Carbon', 'Month_01')
R-squared Score: 0.44959756342468715
RMSE on test set: 224.690923
RMSE on train set: 205.039988
+++++++++++++++++++++++++++++++++++++++++++++++
Features: ('Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity', 'Precipitation', 'Carbon', 'Month_02')
R-squared Score: 0.44461657524594844
RMSE on test set: 224.503669
RMSE on train set: 205.175303
+++++++++++++++++++++++++++++++++++++++++++++++
Features: ('Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity', 'Precipitation', 'Carbon', 'Month_12')
R-squared Score: 0.44104706187587384
RMSE on test set: 224.884510
RMSE on train set: 204.383865
+++++++++++++++++++++++++++++++++++++++++++++++
Features: ('Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity', 'Precipitation', 'Carbon', 'alk/sal')
R-squared Score: 0.4425813397769627
RMSE on test set: 225.584140
RMSE on train set: 205.0

In [114]:
#gradient boosting only combinations of 10 or 11 features
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from itertools import combinations

# Define the target variable and the features you want to use for prediction
target = 'Count'
all_features = ['Latitude', 'Longitude', 'Sodium', 'Temperature', 'pH', 'Alkalinity','Precipitation' ,'Carbon','Month_01','Month_02','Month_12','alk/sal']

max_r2 = 0
best_features = []
best_rmse = 0
best_rmse_train = 0

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[list(all_features)], df[target], test_size=0.3, random_state=42)

# Create a gradient boosting regression model
model = GradientBoostingRegressor(n_estimators=30, max_depth=7)

# Loop over all possible combinations of features
for i in range(10, len(all_features) + 1):
    feature_combinations = combinations(all_features, i)
    for features in feature_combinations:
        # Fit the model to the training data
        model.fit(X_train[list(features)], y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test[list(features)])

        # Evaluate the model's performance using R-squared metric
        r2 = r2_score(y_test, y_pred)
        rmse_test = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        y_pred_train = model.predict(X_train)
        rmse_train = np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred_train)))

        if r2 > 0.4:
            print("Features:", features)
            print("R-squared Score:", r2)
            print("RMSE on test set: {:.6f}".format(rmse_test))
            print("RMSE on train set: {:.6f}".format(rmse_train))
            print("+++++++++++++++++++++++++++++++++++++++++++++++")
        
        # Update the best features and max R-squared score if necessary
        if r2 > max_r2:
            max_r2 = r2
            best_features_r2 = features
            rmse2= rmse_test
            rmse_train1 = rmse_train

# Print the best features and the corresponding R-squared score
print("Features:", best_features_r2)
print("R-squared Score:", max_r2)
print("RMSE on test set: {:.6f}".format(rmse2))
print("RMSE on train set: {:.6f}".format(rmse_train))


c:\Users\chefi\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Month_12
- alk/sal
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 12 features, but GradientBoostingRegressor is expecting 10 features as input.